# Preliminaries

In [2]:
# import ForwardDiff
using PyCall
# import PyPlot
using PyPlot
using ForwardDiff
using DiffBase

pygui(true)



true

# Fun with Hessians

## function x, cost = one_d_minimizer(seed, func; tol=1e-5, maxiter=100, start_eta=0.1)

In [3]:
"""
function x, cost = one_d_minimizer(seed, func; tol=1e-5, maxiter=100, start_eta=0.1)

Minimizes a 1-d function using constrained Hessian minimization. 
We don't trust the long-range info from the Hessian too much, meaning that there's
a given (adaptive) step size. If Newton's method suggests a step smaller than that step
size, we take it. Otherwise, we only move in the direction of the gradient by the stepsize.

Adaptive step size: Every step that the cost function gets smaller, the step grows by a factor 
of 1.1. If a step would have led to a larger cost function, the step is not taken, and
the size falls by a factor of 2.

PARAMETERS:
===========

seed       A float, the starting point for the minimization

func       A one dimensional function, takes a float and returns a float that represents the current cost.


OPTIONAL PARAMETERS:
====================

tol=1e-5   If a step would lead to a change in cost that is smaller in magnitude than tol, stop the minimization.

maxiter=100   Maximum number of iterations for the minimization

start_eta=0.1  The starting value of the step size


RETURNS:
========

x0     the value of x that minimizes f(x)

cost   The value of f(x0) 

"""
function one_d_minimizer(seed, func; tol=1e-5, maxiter=100, start_eta=0.1)
    eta = start_eta;
    lambdavec = [seed]


    out = DiffBase.HessianResult(lambdavec)
    ForwardDiff.hessian!(out, func, lambdavec)
    cost = DiffBase.value(out)
    grad = DiffBase.gradient(out)
    hess = DiffBase.hessian(out)


    for i in [1:maxiter;]
        h_delta = - grad./hess;
        if abs(h_delta[1]) < eta
            new_lambdavec = lambdavec + h_delta
        else
            new_lambdavec = lambdavec - eta*sign(grad)
        end

        if abs(new_lambdavec[1] - lambdavec[1]) < tol
            break
        end
        
        ForwardDiff.hessian!(out, func, new_lambdavec)
        new_cost = DiffBase.value(out)
        new_grad = DiffBase.gradient(out)
        new_hess = DiffBase.hessian(out)

        if new_cost .< cost
            lambdavec[1] = new_lambdavec[1]
            cost = new_cost;
            grad = new_grad;
            hess = new_hess;

            eta = eta*1.1
        else
            eta = eta/2
        end

        # @printf "%d: cost=%.3f lambda=%.3f\n" i cost lambdavec[1]
    end
    
    return lambdavec[1], cost
end
    

one_d_minimizer

## function constrained_parabolic_minimization(H, G, r; tol=1e-6, min_only=true)

In [4]:
"""
function constrained_parabolic_minimization(H, G, r; tol=1e-6, min_only=true)

Given a Hessian matrix, a gradient vector, and a desired radius from the origin, finds the vector 
that minimizes the parabola defined by the Hessian and the gradient, subject to the constraint that the
vector's length equals the desired radius.

PARAMETERS:
===========

H      A square symmetric matrix. It should have all positive eigenvalues.

G      A vector, length equal to the size(H,2)

r      desired radius

OPTIONAL PARAMETERS:
====================

tol=1e-6        Numerical tolerance on the computations

min_only=true   Return only the minimum, or, if false, all xs for all lambdas that match x'*x = r^2


RETURNS:
========

x        The vector that minimizes 0.5*x'*H*x + x'*G subject to x'*x = r
 
J        0.5*x'*H*x + x'*G at the returned x

lambda   value of the Lagrange multiplier at which the radius constraint is satisfied

c        The squared difference between the length of x and r. Should be small, otherwise somthing went wrong!

"""
function constrained_parabolic_minimization(H, G, r; tol=1e-6, min_only=true, 
    doplot=false, efactor=3.0)

    #  --- First a couple of helper functions ----
    
    """
    function x_of_lambda(lambda)

    Given square matrix H, vector G, and passed scalar lambda, returns the vector x that minimizes
    
    0.5 x'*H*x + x'*G - lambda *x'*x

    """
    function x_of_lambda(lambda)
        return inv(H - lambda*eye(size(H,1)))*(-G)
    end
    
    
    """
    function q(lambda, r)

    Returns the squared difference between r and the norm of x_of_lambda(lambda).
    """
    function q(lambda, r)
        return (r - norm(x_of_lambda(lambda)))^2
    end

    
    # First scan lambda to find good candidates for minimizing the parabolic 
    # surface under the x'*x = r^2 constraint
    L = eig(H)[1]
    L0 = maximum(abs(L))
    lambdas = L0*[-efactor:0.01:efactor;]
    costs = zeros(size(lambdas))
    for i in [1:length(lambdas);]
        try 
            costs[i] = q(lambdas[i], r)
        catch
            costs[i] = Inf
        end
    end
    
    if doplot
        figure(2); clf();
        plot(lambdas, costs, "b.-")
        xlabel("lambda")
        ylabel("cost")
    end

    # Take all candidates where the derivative of costs changes sign 
    # from negative to positive (those would be minima),
    # plus the smallest and the largest lambdas tested, as candidates
    g = append!(prepend!(find(diff(sign(diff(costs))) .> 0.99), [1]), [length(lambdas)])

    lambdas_out = zeros(size(g))
    costs_out   = zeros(size(g))
    for i in [1:length(g);]
        lambdas_out[i], costs_out[i] = one_d_minimizer(lambdas[g[i]], x -> q(x[1], r), start_eta=1, tol=tol)
    end

    # Eliminate any lambdas where x'*x doesn't match our desired value r
    I = find(costs_out .< tol)
    lambdas_out = lambdas_out[I]; costs_out = costs_out[I];
    
    # Eliminate any repeated lambdas, to within the specified numerical tolerance.
    I = setdiff(1:length(lambdas_out), find(diff(lambdas_out) .< tol))
    lambdas_out = lambdas_out[I]; costs_out = costs_out[I];
    
    # Find the parabolic estimate of the cost function at these points
    J  = zeros(size(lambdas_out))
    xs = zeros(length(G), length(lambdas_out))
    for i in [1:length(J);]
        xs[:,i] = x_of_lambda(lambdas_out[i])
        J[i] = (0.5*xs[:,i]'*H*xs[:,i] + xs[:,i]'*G)[1]
    end

    # Find and return only the x that has the smallest J
    if min_only
        I = indmin(J)    
    else
        I = 1:length(J)
    end
    return xs[:,I], J[I], lambdas_out[I], costs_out[I]
end

    
x, J, lo, co = constrained_parabolic_minimization([3 2; 2 3], [15,20], 0.25)


([-0.148833,-0.20087],-6.096351909334731,-95.08495660219029,7.932154938695469e-23)

In [5]:
constrained_parabolic_minimization(hess, grad'', eta, doplot=true, efactor=910.0)
ylim([-1, 4])

LoadError: UndefVarError: hess not defined

In [6]:
ylim(-0.01, 0.1)

(-0.01,0.1)

In [7]:
global H = [3 2; 2 3];
global G = reshape([5,7], 2, 1)

function x_of_lambda(lambda)
    global H
    global G
    return inv(H - lambda*eye(size(H,1)))*(-G)
end


# figure(1)



x_of_lambda (generic function with 1 method)

# Testing the parabolic minimization

Define your Hessian and Gradient at the top, and watch it minimize.  
The green dot in Fig 1 should come out at the minimum contour, constrained to the red circle

In [91]:
H = [6 5; 5 -6]    # Hessian
G = [20, 15]''    # Gradient    H & G together define the contour surface
# 
# H = [3 2; 2 3]
# G = [10, 15]''
# G = [5, 7]''
G = [12, 15]''
r = 8            # radius of red circle


# -------------

x = [-10:0.2:10;]'
y = [-10:0.2:10;]

X = repmat(x, length(x), 1); 
Y = repmat(y, 1, length(y)); 

Z = zeros(length(x), length(x))
for i in [1:length(x);]
    for j in [1:length(x);]
        myx = [x[i] y[j]]'
        Z[i,j] = (0.5*myx'*H*myx + G'*myx)[1]
    end
end

figure(1); clf();
ax = gca(); # projection="3d")

ax[:contour](Y, X, Z, 200); # , cmap=ColorMap("hot"))

vlines(0, -10, 10)
hlines(0, -10, 10)



angles = [1:360;]
c = zeros(size(angles))
xs = zeros(size(angles))
ys = zeros(size(angles))
for i in angles
    xs[i] = r*cos(i*pi/180)
    ys[i] = r*sin(i*pi/180)
    myx = [xs[i] ys[i]]'
    c[i] = (0.5*myx'*H*myx + G'*myx)[1]
end
plot(xs, ys, "r-")
title("Green is min of contours constrained to red circle")

figure(2); clf()
plot(angles, c, "b.-")
xlabel("angle")
ylabel("J")
title("Cost function J as we go around round the red circle of Fig 1")

x = constrained_parabolic_minimization(H, G, r)[1]
figure(1); 
plot(x[1], x[2], "g.", markersize=20)

1-element Array{Any,1}:
 PyObject <matplotlib.lines.Line2D object at 0x33b542950>

In [ ]:
myx

# Sandlot

In [15]:
"""
"""

function constrained_Hessian_minimization(seed, func; start_eta=10, tol=1e-6, maxiter=400,
    verbose=false)

    params = seed
    eta = start_eta

    out = DiffBase.HessianResult(params)
    ForwardDiff.hessian!(out, func, params)
    cost = DiffBase.value(out)
    grad = DiffBase.gradient(out)
    hess = DiffBase.hessian(out)

    chessdelta = zeros(size(params))

    for i in [1:maxiter;]
        hessdelta  = - inv(hess)*grad
        try
            chessdelta = constrained_parabolic_minimization(hess, grad'', eta)[1]
            jumptype = "not failed"
        catch
            jumptype = "failed"
        end

        if norm(hessdelta) <= eta
            new_params = params + hessdelta
            jumptype = "Newton"
        elseif jumptype != "failed" 
            new_params = params + chessdelta
            jumptype  = "constrained"
        end

        if jumptype != "failed"
            ForwardDiff.hessian!(out, func, new_params)
            new_cost = DiffBase.value(out)
            new_grad = DiffBase.gradient(out)
            new_hess = DiffBase.hessian(out)

            if abs(new_cost - cost) < tol
                break
            end
        end

        if jumptype == "failed" || new_cost >= cost
            eta = eta/2
        else
            eta = eta*1.1
            params = new_params
            cost = new_cost
            grad = new_grad
            hess = new_hess
        end

        if verbose
            @printf "%d: eta=%.3f cost=%.4f  jtype=%s\n" i eta cost jumptype  
        end
    end
    
    return params
end



constrained_Hessian_minimization (generic function with 1 method)

In [86]:
xs = [-4:0.001:4;]
ideal_params = [-1, 3, 1, 4]
sigma = 0.5
a = ideal_params[1]
b = ideal_params[2]
theta = ideal_params[3]
bias = ideal_params[4]

ys = a + b*tanh.((xs-bias)./theta) 
ys += sigma*randn(size(ys))

function energy(params, x, y)
    a = params[1]
    b = params[2]
    theta = params[3]
    bias = params[4]
    
    J = 0
    for i in [1:length(x);]
        J += (y[i] - (a + b*tanh((x[i]-bias)/theta)))^2
    end

    return J
end


params = [1.0, 1, -1, 1]

params = @time(constrained_Hessian_minimization(params, x -> energy(x, xs, ys), verbose=false,
    tol=1e-12))

# figure(2); clf();
# plot(xs', ys', "b.-")

myx = [0:0.01:1;]*(maximum(xs) - minimum(xs)) + minimum(xs)
# plot(myx, params[1] + params[2]*tanh((myx-params[4])/params[3]), "r-")

out = DiffBase.HessianResult(params)
@time(ForwardDiff.hessian!(out, x -> energy(x, xs, ys), params));

out = DiffBase.GradientResult(params)
@time(ForwardDiff.gradient!(out, x -> energy(x, xs, ys), params));



 11.954988 seconds (32.39 M allocations: 3.942 GB, 9.16% gc time)
  0.085113 seconds (54.85 k allocations: 6.255 MB)
 

 0.068754 seconds (61.75 k allocations: 2.710 MB, 6.02% gc time)


In [89]:
function regular_newton_minimization(seed, func; start_eta=0.05, tol=1e-6, maxiter=400, verbose=false)
    params = seed
    eta    = start_eta
    
    out = DiffBase.HessianResult(params)
    ForwardDiff.hessian!(out, func, params)
    cost = DiffBase.value(out)
    grad = DiffBase.gradient(out)
    hess = DiffBase.hessian(out)

    for i in [1:maxiter;]
        @printf "NEED TO FIX: WHAT IF THE PARABOLIC APPROX DOESN'T LEAD TO A MINIMUM?\n"
        new_params = params - eta * inv(hess)*grad
        
        ForwardDiff.hessian!(out, func, new_params)
        new_cost = DiffBase.value(out)
        new_grad = DiffBase.gradient(out)
        new_hess = DiffBase.hessian(out)
        
        if abs(new_cost - cost)<tol
            break
        end
        
        if new_cost >= cost
            eta=eta/2
        else
            eta = eta*1.1
            if ( eta > 1 )
                eta=1
            end
            params = new_params
            cost = new_cost
            grad = new_grad
            hess = new_hess
        end

        if verbose
            @printf "%d: eta=%.3f cost=%.4f params=[" i eta cost 
            for p in [1:length(params);]
                @printf "%.3f" params[p]
                if p<length(params) @printf ", "; end
            end
            @printf "]\n"
        end
    end
   
    return params
end


regular_newton_minimization (generic function with 1 method)

In [29]:
function adaptive_gradient_minimization(seed, func; start_eta=0.1, tol=1e-6, maxiter=400,
    verbose=false)
    
    params = seed
    eta = start_eta

    out = DiffBase.GradientResult(params)
    ForwardDiff.gradient!(out, func, params)
    cost = DiffBase.value(out)
    grad = DiffBase.gradient(out)

    for i in [1:maxiter;]
        new_params = params - eta*grad

        ForwardDiff.gradient!(out, func, new_params)
        new_cost = DiffBase.value(out)
        new_grad = DiffBase.gradient(out)

        if abs(new_cost - cost) < tol
            break
        end
    
        if new_cost >= cost
            eta = eta/2
        else
            eta = eta*1.1
            params = new_params
            cost = new_cost
            grad = new_grad
        end

        if verbose
            @printf "%d: eta=%.3f cost=%.4f\n" i eta cost  
        end
    end
    
    return params
end


adaptive_gradient_minimization (generic function with 4 methods)

In [72]:
figure(2); clf()

params = [1.0, 1, -1, 1]

params = @time(adaptive_gradient_minimization(params, x -> energy(x, xs, ys), verbose=false,
tol=1e-12, maxiter=2000))


  0.325773 seconds (4.97 M allocations: 240.401 MB, 13.46% gc time)


4-element Array{Float64,1}:
  -5.61208
  -2.07047
  -6.5679 
 -11.1208 

In [83]:
params = [1.0, 1, -1, 1]

params = @time(adaptive_gradient_minimization(params, x -> energy(x, xs, ys), verbose=false,
tol=1e-12, maxiter=24000))

# figure(2); clf();
# plot(xs', ys', "b.-")

myx = [0:0.01:1;]*(maximum(xs) - minimum(xs)) + minimum(xs)
# plot(myx, params[1] + params[2]*tanh((myx-params[4])/params[3]), "r-")

out = DiffBase.GradientResult(params)
# @time(ForwardDiff.gradient!(out, x -> energy(x, xs, ys), params));


 30.445459 seconds (577.86 M allocations: 27.269 GB, 16.24% gc time)


In [31]:
figure(2); clf();
plot(xs', ys', "b.-")

myx = [0:0.01:1;]*(maximum(xs) - minimum(xs)) + minimum(xs)
plot(myx, params[1] + params[2]*tanh((myx-params[4])/params[3]), "r-")

1-element Array{Any,1}:
 PyObject <matplotlib.lines.Line2D object at 0x322010e50>

In [ ]:
lambdas = setdiff([-10.001:0.01:10;], eig(H))
Js = zeros(size(lambdas))
for i in [1:length(lambdas);]
    try
        Js[i] = (norm(x_of_lambda(lambdas[i]))-5)^2
    catch
        println(i)
    end
end

figure(2); clf();
plot(lambdas, Js)
ylim(-10, 20)

In [85]:
include("hessian_utils.jl")

adaptive_gradient_minimization (generic function with 4 methods)

In [ ]:
seed = 1.3
eta = 0.1;
lambdavec = [seed]


out = DiffBase.GradientResult(lambdavec)
ForwardDiff.gradient!(out, x -> q(x[1], 5), lambdavec)
cost = DiffBase.value(out)
grad = DiffBase.gradient(out)


for i in [1:100;]
    new_lambdavec = lambdavec - eta*grad;

    ForwardDiff.gradient!(out, x -> q(x[1], 5), new_lambdavec)
    new_cost = DiffBase.value(out)
    new_grad = DiffBase.gradient(out)

    if new_cost < cost && new_lambdavec[1]>1.0001 && new_lambdavec[1]<4.999 
        lambdavec = new_lambdavec
        cost = new_cost
        grad = new_grad
        eta = eta*1.1
    else
        eta = eta/2
    end

    # @printf "%d: eta=%g cost=%.3f lambda=%.3f\n" i eta cost lambdavec[1]
end

In [ ]:
J = zeros(2,1)

J[1] = size(x[:,1]'*H*x[:,1] + x[:,1]'*G)

In [ ]:
L = eig(H)[1]
r = 0.25

L0 = abs(maximum(L))
lambdas = L0*[-3:0.01:3;]
costs = zeros(size(lambdas))
for i in [1:length(lambdas);]
    try 
        costs[i] = q(lambdas[i], r)
    catch
        costs[i] = Inf
    end
end

figure(2); clf(); plot(lambdas, costs, "-"); ylim(-10, 20); grid()
g = [1 find(diff(sign(diff(costs))) .> 0.99) length(lambdas)]
println(lambdas[g])
vlines(lambdas[g], ylim()[1], ylim()[2])

for i in [1:length(g);]
    lambda, cost = one_d_minimizer(lambdas[g[i]], x -> q(x[1], r), start_eta=1)
    @printf "%d: lambda=%g, cost=%g\n" i lambda cost
end


In [ ]:
lambda, cost = minimizer(20.0, 0.25)

In [ ]:
norm(x_of_lambda(39.5388))

In [ ]:
a = [2, 3;]

b = [1; a; 4]

In [ ]:
h = plot(lambdas+lambdavec[1], 0.5*hess[1]*lambdas.^2 + grad[1]*lambdas + cost, "r-")